+ [paper](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)
+ [Reference](https://github.com/jaewonlee-728/fastcampus-RecSys/blob/master/04-Recommender-System-with-DeepLearning/01-%EB%94%A5%EB%9F%AC%EB%8B%9D%EA%B3%BC%20%EC%B6%94%EC%B2%9C%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98-05-Factorization%20Machine%20%EC%8B%A4%EC%8A%B5.ipynb)

## Importing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import scipy
import math
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [4]:
RANDOM_STATE = 918

In [5]:
data_path = '/content/drive/MyDrive/GH x RippleAI/Dataset/movielens/fastcampus-movielens'
# %cd $data_path

In [6]:
def print_head(df, df_name):
    print( f'------------- {df_name + str(df.shape):^20} -------------')
    print()
    print(df.head(), '\n')

In [7]:
ratings_df = pd.read_csv(os.path.join(data_path, 'ratings.csv'))
movies_df = pd.read_csv(os.path.join(data_path, 'movies.csv'))
tags_df = pd.read_csv(os.path.join(data_path, 'tags.csv'))

print_head(ratings_df, 'ratings_df')
print_head(movies_df, 'movies_df')
print_head(tags_df, 'tags_df')

------------- ratings_df(100836, 4) -------------

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931 

-------------  movies_df(9742, 3)  -------------

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy   

-------------   tags_df(3683, 4)   -------------

   userId  movieId    

## Featrue Engineering

+ userId one hot vector
+ movieId one hot vector
+ other movie rated (sum up to 1)
+ time stamp (x)
+ tags one hot vector ( +@ )
+ genres one hot vector ( +@ )


In [8]:
X_temp = ratings_df.groupby(['userId', 'movieId'])['rating', 'timestamp'].mean().reset_index()
print(X_temp.head())



   userId  movieId  rating    timestamp
0       1        1     4.0  964982703.0
1       1        3     4.0  964981247.0
2       1        6     4.0  964982224.0
3       1       47     5.0  964983815.0
4       1       50     5.0  964982931.0


In [27]:
# userId
userId_encodded = pd.get_dummies(X_temp['userId'], prefix = 'user_')
assert(userId_encodded.shape[0] == X_temp.shape[0])
assert(userId_encodded.shape[1] == X_temp['userId'].nunique())
print(userId_encodded.shape)

# movieId
movieId_encodded = pd.get_dummies(X_temp['movieId'], prefix = 'movie_')
assert(movieId_encodded.shape[0] == X_temp.shape[0])
assert(movieId_encodded.shape[1] == X_temp['movieId'].nunique())
print(movieId_encodded.shape)

# other_rated

# time은 그냥 빼버리기 
# time_encodded = X_temp['timestamp']


X_simple = pd.concat([userId_encodded, movieId_encodded], axis = 1)
print(X_simple.shape)



(100836, 610)
(100836, 9724)
(100836, 10334)


In [ ]:
ratings_df['movieId'].nunique()

9724

## Factorization Implementation

+ Predict Function
> $\widehat{y}(x) = w_{0} +  \sum_{i=1}^{n} w_{i}x_{i} +  \sum_{i=1}^n \sum_{j=+1}^n <\overrightarrow v_{i}, \overrightarrow v_{j}>x_{i}x_{j}$  
$w_{0} \in \mathbf{R}, w \in \mathbf{R^n}, \overrightarrow v_{i} \in \mathbf{R^k}$ 

+ Real Implementation ( proof in paper )

> $ \sum_{i=1}^{n}\sum_{j=i+1}^{n} <\overrightarrow v_{i}, \overrightarrow v_{j}>x_{i}x_{j} $    
$ = 1/2 * \sum_{f=1}^{k}( (\sum_{i=1}^{n} \overrightarrow v_{i, f}x_{i})^2 - (\sum_{i=1}^{n}v_{i}^2x_{i}^2 )  )$

+ Gradient

> $ \partial w0 = \hat{y} - y$  
$ \partial w_{i} = x_{i}(\hat{y} - y) + \lambda _{1} * w_{i} $    
$ \partial v_{i,f} = (\hat{y} - y)(x_{i} \sum_{i=1}^{n} v_{i,f}x_{i} - x_{i}^2v_{i,f}) + \lambda2  $


+ csr_matrix  
indptr[i] : first ith row elements index   
indices[i] : ith element's col index

+ Brief arguments  

    X : csr_matrix   
    y : nd.array   
    n_samples : row   
    n_features : col   
    v : nd.array(n_factors, n_samples) (each column vectors)  
    w0, w : coef   

In [99]:
def predict(X, w0, w, v, n_factors, i):
    """
    Argument:
    X -- csr_matrix
    i -- ith sample
    w0, w, v : coef ((n,1), (n_factors, ), (k, n_features))
    n_factors -- latent_dimension
    
    Returns:
    pred -- predicted rating (scalar)
    summed - sum of (vi * xi), for gradient caculate 
    """
    data, indptr, indices = X.data, X.indptr, X.indices

    # see above formula
    summed = np.zeros(n_factors)
    summed_squared = np.zeros(n_factors)

    pred = w0[0] # bias

    # degree 1
    for index in range(indptr[i], indptr[i+1]):
        col = indices[index]
        pred += w[col] * data[index]
    # degree 2 (only for nonzeros)
    for factor in range(n_factors):

        for index in range(indptr[i], indptr[i+1]):
            col = indices[index]
            term = v[factor, col] * data[index]
            summed[factor] += term
            summed_squared[factor] += term * term
        
        pred += 0.5 * (summed[factor] * summed[factor] - summed_squared[factor])

    return pred, summed

In [74]:
def sgd(X, y, n_samples, n_features, w0, w, v, 
        n_factors, learning_rate=0.01, reg_w=0.01, reg_v=0.01):
    """
    1 epoch for each function call
    Stochastic Gradient Descent

    Argument:
    X -- csr_matrix
    y -- real y 
    n_samples, n_features
    w0, w, v : coef ((1,), (n_factors, 1), (n_factors, n_features))
    n_factors -- latent_dimension
    learning_rate, reg_w, reg_v -- for learning and regularization
    
    Returns:
    loss - return mean loss after iterating one epoch
    """
    data, indptr, indices = X.data, X.indptr, X.indices
    loss = 0.0

    # roof for each instances
    for i in range(n_samples):
        pred, summed = predict(X, w0, w, v, n_factors, i)

        # squared error 
        diff = pred - y[i]
        loss += 0.5 * diff**2

        # bias
        w0[0] -= learning_rate * diff

        # update w
        for index in range(indptr[i], indptr[i+1]):
            col = indices[index]
            w[col] -= learning_rate * (diff*data[index] + reg_w*w[col])

        # update v
        for factor in range(n_factors):
            for index in range(indptr[i], indptr[i+1]):
                col = indices[index]
                # summed(vi*xi) - xi * vif
                term = summed[factor] - data[index] * v[factor, col]
                v_gradient = diff * data[index] * term
                v[factor, col] -= learning_rate * (v_gradient + reg_v * v[factor, col])
                                                   
    loss /= n_samples
    return loss

In [78]:
def fit(X, y, config):
    """
    Argument:
    X -- csr_matrix
    y -- real y 
    config -- dictionary {
        "num_epochs" : ?,
        "num_factors" = ?,
        "learning_rate" = ?,
        "reg_weights" = ?,
        "reg_features" = ?
        }

    
    Returns:
    epoch_loss -- return loss list for each epoch
    params -- dictionary {
      "w0" : (1,) for mutability
      "w"  : (n_factors, )
      "v"  : (n_factors, n_features) 
    }
    """



    epochs = config['num_epochs']
    num_factors = config['num_factors']
    learning_rate = config['learning_rate']
    reg_weights= config['reg_weights']
    reg_features = config['reg_features']

    num_samples, num_features = X.shape
    w = np.zeros(num_features,) 
    w0 = np.array([1.0])
    v = np.random.randn(num_factors, num_features)

    epoch_loss = []
    for epoch in range(epochs):
        loss = sgd(X, y, num_samples, num_features,
                   w0, w, v, num_factors, learning_rate,
                   reg_weights, reg_features)
        print(f'[epoch : {epoch+1}], loss : {loss:.5f}, {w0[0]:.4f}')
        epoch_loss.append(loss)
        
    params = {"w0" : w0, "w" : w, "v" : v}
    return epoch_loss, params

## Test

In [76]:
config = {
    "num_epochs": 10,
    "num_factors": 10,
    "learning_rate": 0.1,
    "reg_weights": 0.01,
    "reg_features": 0.01
}


### Simple

In [ ]:
X_train_simple, X_test_simple, y_train_simple, y_test_simple = \
  train_test_split(X_simple, X_temp['rating'], test_size = 0.2, random_state = RANDOM_STATE )

X_train_simple_sparse = scipy.sparse.csr_matrix(X_train_simple.values)
X_test_simple_sparse = scipy.sparse.csr_matrix(X_test_simple.values)
epoch_loss, params = fit(X_train_simple_sparse, y_train_simple.values, config)

test_case = X_test_simple.shape[0]
preds = np.zeros(test_case)



In [101]:
for i in range(test_case):
  preds[i], temp  = predict(X = X_test_simple_sparse, n_factors = config["num_factors"] , 
                     w0 = params["w0"], w= params["w"], v = params["v"],i = i )

rmse = np.sqrt( mean_squared_error(preds, y_test_simple.values) )
print(f'RMSE : {rmse:.4f}')

RMSE : 1.1960
